<a href="https://colab.research.google.com/github/emelex-ai/ConnTextUL/blob/main/MCB_%26_WandB_ConnTextUL_Model_(in_progress).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Datasets

### Get Data

Load some data into the content folder. This should be in our shared ConnTextUL folder. You may need to move the shared folder to a location in your drive with the same full path as indicated here. Or, we can devise a more efficient way to share data in the future.

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

I_AM = 'Nathan'
#I_AM = 'Tyler'

if I_AM == 'Nathan':
  drive.mount('/gdrive', force_remount=True)
  !ln -s "/gdrive/My Drive/Projects/Modeling Reading Programs/ConnTextUL/data" "/content"
  DATA_PATH = "/content/data/data.csv"
elif I_AM == 'Tyler':
  drive.mount('/content/Hootie_Hoo')
  DATA_PATH = "/content/Hootie_Hoo/MyDrive/data_A.csv"

Mounted at /gdrive
ln: failed to create symbolic link '/content/data': File exists


# ConnTextUL Reading Model 001 (*with WandB logging & MCB's encoding*)

## Change Log

-  (5/15/2023)
   * Decoder inputs (necesary for teacher forcing) are now embedded properly in the *forward* function. This should resolve the attn_mask typing error.

   --- Luke
-  (05/14/2023)
   * Modified both the character tokenizer and the phonology tokenizer to return more elaborate dictionaries with all of the input and masks needed for the encoders and decoders
   * Completed a comprehensive walk through with examples for the orthography -> orthography pathway and the phonology -> phonology pathway in the **Examples** section at the bottom of the notebook.
   * TODO: Take everything in the **Examples** section and put it in the appropriate places in the model and the training loop.
   * ALMOST THERE!

   --- Nathan
-  (05/13/2023)
   * Began to modify the phonemizer class and the dataset class to carry a _targets_ object that contains the binary class labels for the CrossEntropy criterion. 
   * TODO: We need to check the shapes of all objects being passed into the the CrossEntropy criterion and ensure that they all conform to the requirements specified in the documentation. This may include adding some additional padding. The necessary reshaping of the phonological vectors make reasoning in this higher dimensional space a bit difficult.
   
   ---Nathan
-  (05/12/2023)
   * Made numerous code refactoring changes
   * Implemented the phonology_tokenizer and integrated it into the dataset class
   * Reran the training loop code and worked out many bugs associated with shapes and batch size idiosyncrasies
   * Added various try/except blocks and assert statements for shape and type constraints
   * TODO: Finish the cross entropy loss function. Rerun the whole notebook and just keep rerunning the training loop and ironing out any remaining bugs
   * NOTE: We need to think through carefully about how we are averaging the phonological vectors and whether to reinstert them as row or column vectors in the final phonological_input_tensor

   --- Nathan
-  (05/08/2023)
   * Adjusted `__getitem__` method of `ConnTextUL` Dataset so that for batches (called with slice object), it returns a pair whose phoneme entry is an array (the batch) of phoneme arrays (corresponding to words) with SOS, EOS, and PAD index lists at front and back of array.

   --- Tyler

-  (05/07/2023)
   * Finished/re-wrote `ConnTextUL` Dataset. I finished it once, but found that it would take 24+ hours to construct, because I had it querying CMUDict word-by-word. So I re-wrote the class to query CMUDict in a single batch. Because the `Traindata.traindata` dictionary re-orders/re-organizes all the phonological feature vectors differently than they would be ordered according to the input word list, I had to include some secondary re-organization to get the data back into the original input order. **One potential problem:** The `Traindata` constructor drops single-letter words and words with punctuation. The `__getitem__` method for the `ConnTextUL` class currently outputs a tuple consisting of a word string and its associated list-of-lists of the indices of nonzero entries in the phoneme feature vectors associated to the word. I included a instance of the `CharacterTokenizer` in the `ConnTextUL` class in case we want get item to output tokenized word instead of original word string. 

   --- Tyler

-  (05/01/2023)
   * Implemented the data ingestion/transformation code for training on the csv
     file produced by Alia Pugh. 
   * Fixed the _init_ function in the CharacterTokenizer. Also, the encoder was using 
     [BOW], [EOW] but the _init_ function initialized [BOS], [EOS].
   * Updated the encode routine in the CharacterTokenizer to add the [EOS] token before
     the padding.
   * Changed the GraphoneDataset class to be ConnTextULDataset class and incorporated
     Alia's dataset and began incorporating Matt's Phonological vectors.

   --- Nathan

-  (4/29/2023)
   *  Cloned Matt Cooper-Borkenhagen's Traindata repositroy and imported
      associated library.
   *  Did a little bit of testing to make sure I understand the dictionary
      `Traindata.traindata` attribute of MCB's Traindata class.
   *  Added testing and then comments throughout the notebook to make sure I
      understand the model sturcture, in order to make necessary changes using
      MCB's library.
   *  Change outline (TODO): Replace phoneme tokensizer with a module that reutns an
      array of MCB's vectors for each word, embeds these (linearly?) in a space of
      dimension `d_model`, and then takes average per word.
      HOW TO DO THIS: Redefine the phoneme embedding at the very beginning of GraphoneModel
      to be a linear embedding from whatever space MCB avg-sums live into a space
      of dimension d_model.

   --- Tyler


-  (4/25/2023)
   *  Included WandB functionality throughout. Notebook now logs data in our
      WandB `emelex/ConnTextUL_WandB_trial` project.

   --- Tyler


-  (4/2/2023)
   * Implemented global attention for producing orthography/phoneme embedding. The previous
     approach (concatenate all timesteps into a single vector + feedforward) was
     memory-intensive and risked contaminating the embeddings w/ 
     padding.
   * Removed some redundant layers in both encoder and decoder
   * _generate_ still doesn't work; need to discuss tokenizer w/ group

   --- Luke 


- (4/1/2023)
  * Spin up Spot Instance with hosted notebook: Nathan
  * Training Loop must be completed: Luke
  * Finish the Generate Function
  * Tensorboard probably to be used in some capacity: Nathan/Luke in the future

  --- Nathan

## Dependencies and Monitoring

### Import basic libraries

In [1]:
from torch.utils.data import Dataset
import pandas as pd
import torch as pt
pt.manual_seed(57713)
import numpy as np
import math
import random
import tqdm
import sys

import contextlib
from io import StringIO

___

### Cloning/importing MCB's `Traindata` library.

In [ ]:
!apt-get install -y git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.25.1-1ubuntu3.11).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
!git clone https://github.com/MCooperBorkenhagen/Traindata

fatal: destination path 'Traindata' already exists and is not an empty directory.


Checking which of the library's dependencies are already present:

In [ ]:
!pip freeze | grep 'numpy\|json\|nltk\|copy\|string\|utilities'

fastjsonschema==2.16.3
jsonpickle==3.0.1
jsonschema==4.3.3
nltk==3.8.1
numpy==1.22.4


In [ ]:
%cd Traindata

/content/Traindata


Import MCB's `Traindata` library:

In [ ]:
import Traindata
from Traindata import Traindata

In order to use the `Traindata` library, we need the resourse `cmudict`. This is an online dictionary called the *The Carnegie Mellon University Pronouncing Dictionary*, which provides a standardized phonetic 'pronounciation' of words in a 134,000-word set. The following cell obtains this resource.

In [ ]:
import nltk
nltk.download('cmudict')

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


True

___

### Setup WandB
These next 2 cells install WandB and connect the present notebook to WandB for logging and tracking.

In [ ]:
!pip install wandb -qU

In [ ]:
# Log in to your W&B account
import wandb
wandb.login()

wandb: Currently logged in as: nathan-crock (emelex). Use `wandb login --relogin` to force relogin


True

____

### Tokenizers

**Endpoints:** The `encode` method of the `CharacterTokenizer` class takes as input a list of strings. I'm uncertain about requirements on the strings in this list. It seems to be fine if the strings are nonsense words or contain numerical characters.

If we input a list of length $N$, then the `encode` method of this class outputs a tensor fo size $N\!\times\!M$, where $M$ is related to the maximum length of the strings in our input list. That said, *it's not clear to me exactly how the dimension value $M$ is determined using the lengths of the strings in our list.*

**Note.** Comments in cell immediately below explain what the `encode` method does. -- Tyler

In [ ]:
class CUDA_Dict(dict):

  def to(self, device):

    output = {}
    for key in self.keys():
      batches = self[key]
      if isinstance(batches, list):
        try:
          output[key] = [[val.to(device) for val in batch] for batch in batches]
        except:
          print(f"batches = {batches}")
          raise
      elif isinstance(batches, pt.Tensor):
        output[key] = batches.to(device)
      else:
        raise TypeError("Must be list or torch tensor")

    return output

#### Character Tokenizer

In [ ]:
class CharacterTokenizer:
    def __init__(self, list_of_characters):

        # Include custom tokens into vocabulary
        self.vocab = ['[BOS]', '[EOS]', '[CLS]', '[UNK]', '[PAD]']
        self.vocab.extend(list_of_characters)

        self.char_2_idx, self.idx_2_char = {}, {}
        for i, ch in enumerate(self.vocab):
          self.char_2_idx[ch] = i 
          self.idx_2_char[i] = ch
        # Reuse index from previous for loop to save computation
        self.size = i+1


    def __len__(self): return self.size


    def encode(self, list_of_strings):
        assert isinstance(list_of_strings, str) or (isinstance(list_of_strings, list) \
                 and all(isinstance(string, str) for string in list_of_strings))
        if isinstance(list_of_strings, str): list_of_strings = [list_of_strings]

        lengths = [len(string) for string in list_of_strings]
        max_length = max(lengths)

        # Padding function, puts beginning-of-string and end-of-string tokens
        # on beginning and end of string, after padding the original string
        # up to max length of string.
        # This function converts the output to a list rather than string.
        enc_pad = lambda string: ['[BOS]'] + list(string) + ['[EOS]'] + (max_length - len(string)) * ['[PAD]']
        dec_pad = lambda string: ['[BOS]'] + list(string) + (max_length - len(string)) * ['[PAD]']

        list_of_enc_strings = list(map(enc_pad, list_of_strings))
        list_of_dec_strings = list(map(dec_pad, list_of_strings))

        # Initiate encoder tokens tensor as tensor of zeros, 
        enc_input_ids = pt.zeros((len(list_of_enc_strings), 2 + max_length), dtype=pt.long)
        for idx, string in enumerate(list_of_enc_strings):
            for jdx, char in enumerate(string):
                enc_input_ids[idx, jdx] = self.char_2_idx.get(char, 3) # Default to [UNK]

        # Initiate decoder tokens tensor as tensor of zeros, 
        dec_input_ids = pt.zeros((len(list_of_dec_strings), 1 + max_length), dtype=pt.long)
        for idx, string in enumerate(list_of_dec_strings):
            for jdx, char in enumerate(string):
                dec_input_ids[idx, jdx] = self.char_2_idx.get(char, 3) # Default to [UNK]                

        PAD_TOKEN = self.char_2_idx['[PAD]']
        enc_pad_mask = (enc_input_ids == PAD_TOKEN)
        dec_pad_mask = (dec_input_ids == PAD_TOKEN)
        return CUDA_Dict({'enc_input_ids':enc_input_ids,
                          'dec_input_ids':dec_input_ids,
                          'enc_pad_mask':enc_pad_mask.bool(),
                          'dec_pad_mask':dec_pad_mask.bool()})
            

    def decode(self, list_of_ints):
        assert isinstance(list_of_ints,int) or (isinstance(list_of_ints,int) \
                 and all(isinstance(ints,int) for ints in list_of_ints))
        if isinstance(list_of_ints,int): list_of_ints = [list_of_ints]

        outputs = [''.join([self.idx_2_char.get(i) for i in ints]) for ints in list_of_ints]

In [ ]:
# First create the list_of_characters object to initialize the tokenizer
vocab = ['one', 'thing', 'I', 'know', 'two', 'three', 'four', 'what', 'hercules', 'theoretical', 'blegrid', 'phrempral']
list_of_characters = set(''.join([c for word in vocab for c in word])) # Don't use this if vocab is large
character_tokenizer = CharacterTokenizer(list_of_characters)
character_tokenizer.char_2_idx

{'[BOS]': 0,
 '[EOS]': 1,
 '[CLS]': 2,
 '[UNK]': 3,
 '[PAD]': 4,
 'r': 5,
 't': 6,
 'a': 7,
 'e': 8,
 'p': 9,
 'n': 10,
 'f': 11,
 'c': 12,
 'u': 13,
 's': 14,
 'd': 15,
 'h': 16,
 'w': 17,
 'i': 18,
 'l': 19,
 'o': 20,
 'I': 21,
 'g': 22,
 'k': 23,
 'm': 24,
 'b': 25}

In [ ]:
Y = character_tokenizer.encode(['hercules', 'kid', 'a', 'whale'])
enc_input_ids = Y['enc_input_ids']
print(f"enc_input_ids\n{enc_input_ids}")
enc_pad_mask = Y['enc_pad_mask']
print(f"enc_pad_mask\n{enc_pad_mask}")
dec_input_ids = Y['dec_input_ids']
print(f"dec_input_ids\n{dec_input_ids}")
dec_pad_mask = Y['dec_pad_mask']
print(f"dec_pad_mask\n{dec_pad_mask}")

enc_input_ids
tensor([[ 0, 16,  8,  5, 12, 13, 19,  8, 14,  1],
        [ 0, 23, 18, 15,  1,  4,  4,  4,  4,  4],
        [ 0,  7,  1,  4,  4,  4,  4,  4,  4,  4],
        [ 0, 17, 16,  7, 19,  8,  1,  4,  4,  4]])
enc_pad_mask
tensor([[False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False,  True,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False,  True,  True,  True]])
dec_input_ids
tensor([[ 0, 16,  8,  5, 12, 13, 19,  8, 14],
        [ 0, 23, 18, 15,  4,  4,  4,  4,  4],
        [ 0,  7,  4,  4,  4,  4,  4,  4,  4],
        [ 0, 17, 16,  7, 19,  8,  4,  4,  4]])
dec_pad_mask
tensor([[False, False, False, False, False, False, False, False, False],
        [False, False, False, False,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True,  True],
        [False, Fals

#### Phonology Tokenizer

In [ ]:
# We may want to calculate all of the phonological vectors for the entire dataset ahead of time here.
# This way, we can just look up the vector instead of calculating it on the fly. That will save time
# during training.

class Phonemizer():

  def __init__(self, wordlist):

    self.PAD = 33

    traindata = Traindata(wordlist, 
                          phonpath='raw/phonreps.csv',  
                          terminals=True,
                          oneletter=True,
                          verbose=False).traindata
    self.enc_inputs = {}
    self.dec_inputs = {}
    self.targets = {}
    for length in traindata.keys():
      for word_num, (phon_vec_sos, phon_vec_eos) in enumerate(zip(traindata[length]['phonSOS'], traindata[length]['phonEOS'])):
        word = traindata[length]['wordlist'][word_num]
        # The encoder receives the entire phonological vector include the BOS and EOS tokens
        self.enc_inputs[word] = [pt.tensor(np.where(vec)[0], dtype=pt.long) for vec in phon_vec_sos] + [pt.tensor([32])] # 32 is the EOS token location
        # The decoder received the entire phonological vectors including the BOS token, but not the EOS token
        self.dec_inputs[word] = [pt.tensor(np.where(vec)[0], dtype=pt.long) for vec in phon_vec_sos]
        # The target for the decoder is all phonological vectors including the EOS token, but excluding the BOS token
        self.targets[word] = phon_vec_eos

    del traindata

  def __len__(self):
    
    return 34

  def encode(self, wordlist):

    enc_input_ids = []
    dec_input_ids = []
    targets = []

    if isinstance(wordlist, list):
      max_length = 0
      for word in wordlist:
        # Make sure all words are in the phonological dictionary
        enc_input = self.enc_inputs.get(word, None)
        dec_input = self.dec_inputs.get(word, None)
        target = self.targets.get(word, None)
        # If any word is not in the dictionary, skip the batch
        if enc_input is None or dec_input is None or target is None:
          return None
        # Collect all token lists in a larger list while calculating the max length of this batch
        enc_input_ids.append(enc_input.copy())
        dec_input_ids.append(dec_input.copy())
        targets.append(pt.tensor(target.copy(), dtype=pt.long))
        # All three, enc_input, dec_input, and target should be the same length. So all share the same max_length
        # (though we subtract 1 from the decoder input and targets because the BOS/EOS tokens were removed)
        max_length = max(max_length, len(enc_input))
      # Now that we know the max length of this batch, we pad the encoder and decoder input token list with PAD tokens
      for epv, dpv in zip(enc_input_ids, dec_input_ids):
        epv.extend([pt.tensor([self.PAD])]*(max_length-len(epv)))
        dpv.extend([pt.tensor([self.PAD])]*(max_length-1-len(dpv)))
      # We then include padding, or indices in the targets to be passed to the 'ignore_index' parameter in the CrossEntropyLoss
      # Since each phonological vector is either on or off, it is a binary classification problem, so valid labels are either 0, or 1.
      # We will include labels of '2' where the padding is in the target vectors
      #print("targets = ", targets)
      for i in range(len(targets)):
        tv = targets[i]
        targets[i] = pt.concat((tv, pt.tensor([[2]*33]*(max_length-1-len(tv)), dtype=pt.long)))
        #print("len(tv) = ", len(tv))
        #tv = pt.concat((tv, pt.tensor([[2]*33]*(max_length-len(tv)))))
        #print("tv = ", tv)
        #sys.exit()
    else:
      raise TypeError('encode only accepts lists or a single string as input')

    enc_pad_mask = pt.tensor([[all(val == pt.tensor([self.PAD])) for val in token] for token in enc_input_ids])
    dec_pad_mask = pt.tensor([[all(val == pt.tensor([self.PAD])) for val in token] for token in dec_input_ids])
    #dec_pad_mask = pt.tensor([1])

    # Ensure that the number of tokens matches the number of boolean values in the mask
    assert len(enc_input_ids) == len(enc_pad_mask), f"tokens is length {len(enc_input_ids)}, enc_pad_mask is length {len(enc_pad_mask)}. They must be equal"

    # Do we need to pad the targets? We do to convert it to a tensor which is needed for the CrossEntropy criterion
    return CUDA_Dict({'enc_input_ids':enc_input_ids, 
                      'enc_pad_mask':enc_pad_mask.bool(), 
                      'dec_input_ids':dec_input_ids,
                      'dec_pad_mask':dec_pad_mask.bool(),
                      'targets':pt.stack(targets, 0)})

In [ ]:
dataset = pd.read_csv(DATA_PATH)
words = dataset['word_raw'].str.lower()

In [ ]:
phonemizer = Phonemizer(words)

In [ ]:
val = phonemizer.encode(['whale', 'hi']); val

{'enc_input_ids': [[tensor([31]),
   tensor([ 0, 11]),
   tensor([14, 15, 18, 19, 24, 27, 29]),
   tensor([ 2, 10, 14]),
   tensor([32])],
  [tensor([31]),
   tensor([5, 7]),
   tensor([14, 15, 23, 27, 29]),
   tensor([32]),
   tensor([33])]],
 'enc_pad_mask': tensor([[False, False, False, False, False],
         [False, False, False, False,  True]]),
 'dec_input_ids': [[tensor([31]),
   tensor([ 0, 11]),
   tensor([14, 15, 18, 19, 24, 27, 29]),
   tensor([ 2, 10, 14])],
  [tensor([31]), tensor([5, 7]), tensor([14, 15, 23, 27, 29]), tensor([33])]],
 'dec_pad_mask': tensor([[False, False, False, False],
         [False, False, False,  True]]),
 'targets': tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
           0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
          [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 

In [ ]:
val['targets'].shape

torch.Size([2, 4, 33])

### ConnTextUL Dataset







In [ ]:
class ConnTextULDataset(Dataset):
  """ConnTextULDataset

  Dataset of 

  For Matt's Phonoligical Feature Vectors, we will use (31, 32, 33) to represent ('[BOS]', '[EOS]', '[PAD]')

  """
  def __init__(self):

      self.dataset = pd.read_csv(DATA_PATH)
      tmp_words = self.dataset['word_raw'].str.lower() # Series of all lowercased words
      self.phonology_tokenizer = Phonemizer(tmp_words)
      # self.listed_words = [word for word in self.words]

      # Notice I created a tokenizer in this class.
      # We can use it to tokenize word output of __getitem__ below,
      # although I haven't implemented yet.
      list_of_characters = set(''.join([c for word in tmp_words for c in word]))
      self.character_tokenizer = CharacterTokenizer(list_of_characters)

      final_words = []
      for word in tmp_words:
        if word == "" or word == None or word == []:
          continue
        if self.phonology_tokenizer.encode([word]): #check if in phoneme_dict
          final_words.append(word)

      self.words = final_words

          
  def __len__(self):
      length = len(self.words)  
      return length  

  def __getitem__(self, idx):

      string_input = self.words[idx]
      # tokenized orthographic output. character_tokenizer pads and wraps in tensor
      orth_tokenized = self.character_tokenizer.encode(string_input)
      phon_tokenized = self.phonology_tokenizer.encode(string_input)

      return {'orthography': orth_tokenized, 'phonology': phon_tokenized}

Testing:

*The construciton of the `ConnTextULDataset` instance here takes about 30 seconds.*

In [ ]:
conntextul_dataset = ConnTextULDataset()
length = len(conntextul_dataset)
print('\nDataset length:', length)

In [ ]:
print('\nSome random examples, in the form (word, phoneme_feature_indices)\n')
NUM = 2
for _ in range(NUM):
  start = random.randrange(length-NUM)
  idx = slice(start, start+NUM) 
  orth, phon = conntextul_dataset[idx]
  print(f"\nconntextul_dataset[{idx}]['orthography']['enc_input_ids']\n", conntextul_dataset[idx]['orthography']['enc_input_ids'])
  print(f"\nconntextul_dataset[{idx}]['orthography']['enc_pad_mask']\n", conntextul_dataset[idx]['orthography']['enc_pad_mask'])
  print(f"\nconntextul_dataset[{idx}]['phonology']['enc_input_ids']\n", conntextul_dataset[idx]['phonology']['enc_input_ids'])
  print(f"\nconntextul_dataset[{idx}]['phonology']['enc_pad_mask']\n", conntextul_dataset[idx]['phonology']['enc_pad_mask']) 
  #print('Dataset item {}:'.format(idx), conntextul_dataset[idx])



Some random examples, in the form (word, phoneme_feature_indices)


conntextul_dataset[slice(29425, 29427, None)]['orthography']['enc_input_ids']
 tensor([[ 0,  8, 36, 43, 46,  9,  1],
        [ 0, 24, 26, 12,  1,  4,  4]])

conntextul_dataset[slice(29425, 29427, None)]['orthography']['enc_pad_mask']
 tensor([[False, False, False, False, False, False, False],
        [False, False, False, False, False,  True,  True]])

conntextul_dataset[slice(29425, 29427, None)]['phonology']['enc_input_ids']
 [[tensor([31]), tensor([14, 16, 21]), tensor([ 0, 11]), tensor([14, 17, 19, 24, 26, 28, 29]), tensor([4, 6]), tensor([32])], [tensor([31]), tensor([4, 6]), tensor([14, 16, 21, 29]), tensor([0, 6]), tensor([32]), tensor([33])]]

conntextul_dataset[slice(29425, 29427, None)]['phonology']['enc_pad_mask']
 tensor([[False, False, False, False, False, False],
        [False, False, False, False, False,  True]])

conntextul_dataset[slice(63034, 63036, None)]['orthography']['enc_input_ids']
 tensor([[ 

## The Model

In [ ]:
d_model=512

### Encoder module
**Endpoints:** The `Encoder` module takes in a torch tensor of size either $N\!\times\!d_{model}$ or $B\!\times\!N\!\times d_{model}$, where $B$ denotes batch size and $N$ denotes sequence length, and then outputs a torch tensor of corresponding size $N\!\times\!d_{model}$ or $B\!\times\!N\!\times d_{model}$.

In [ ]:
class Encoder(pt.nn.Module):
    def __init__(self, d_model=512, nhead=1, num_layers=1):
        super(Encoder, self).__init__()
        encoder_layer = pt.nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer_encoder = pt.nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        output = self.transformer_encoder(src, mask=src_mask, src_key_padding_mask=src_key_padding_mask)
        return output

### Decoder module
**Endpoints:** The `Decoder` module takes in a pair of torch tensors. The first tensor argument, denoted `tgt` in the code below, is the sequence to the decoder. The second tensor argument, denoted `memory` in the code below, is  the sequence from the last layer of the encoder.

Each of the `tgt` and `memory` arguments is a torch tensor of size either $N\!\times\!d_{model}$ or $B\!\times\!N\!\times d_{model}$, where $B$ denotes batch size and $N$ denotes sequence length. The `Decoder` module outputs a torch tensor of corresponding size $N\!\times\!d_{model}$ or $B\!\times\!N\!\times d_{model}$.

In [ ]:
class Decoder(pt.nn.Module):
    def __init__(self, d_model=512, nhead=1, num_layers=1):
        super().__init__()
        decoder_layer = pt.nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer_decoder = pt.nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        output = self.transformer_decoder(tgt, memory, 
                                            tgt_mask=tgt_mask, 
                                            memory_mask=memory_mask, 
                                            tgt_key_padding_mask=tgt_key_padding_mask, 
                                            memory_key_padding_mask=memory_key_padding_mask)

        return output

Testing:

In [ ]:
decoder = Decoder()

In [ ]:
decoder.transformer_decoder

TransformerDecoder(
  (layers): ModuleList(
    (0): TransformerDecoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
      )
      (multihead_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
      )
      (linear1): Linear(in_features=512, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=512, bias=True)
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm3): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (dropout3): Dropout(p=0.1, inplace=False)
    )
  )
)

In [ ]:
a = pt.rand(64, 512)
b = pt.rand(64, 512)
y = decoder(a, b)
print('\n', type(y))
print(y.shape)

a = pt.rand(64, 10, 512)
b = pt.rand(64, 10, 512)
y = decoder(a, b)
print('\n', type(y))
print(y.size())


 <class 'torch.Tensor'>
torch.Size([64, 512])

 <class 'torch.Tensor'>
torch.Size([64, 10, 512])


### The Model
This is the PyTorch `nn` class that puts everything together to produce our ConnTextUL model. Comments within the code describe how all the class attributes correspond to segments in [our model diagram](https://drive.google.com/file/d/1QVzH68d4qowdYgW5YzQI1ADN-p9OeBDF/view?usp=share_link)

In [ ]:
class Model(pt.nn.Module):
    def __init__(self,
                 orth_vocab_size,
                 phon_vocab_size,
                 d_model=512,
                 nhead=1,
                 num_layers=1,
                 max_seq_len=20):
        super().__init__()

        # Initial embeddings for orthography, phonology, and position
        # Emebdding for orthography
        self.orthography_embedding = pt.nn.Embedding(orth_vocab_size, d_model)
        # Embedding for phonology
        self.phonology_embedding = pt.nn.Embedding(phon_vocab_size, d_model)
        self.position_embedding = pt.nn.Embedding(max_seq_len, d_model)

        self.vocab_sizes = {'orth_vocab_size': orth_vocab_size, 
                            'phon_vocab_size': phon_vocab_size}
        self.d_model = d_model
        self.max_seq_len = max_seq_len
        
        # A 1×1×d_model tensor of model parameters, rescaled by √d_model
        self.global_embedding = pt.nn.Parameter(pt.randn((1,1,self.d_model))/self.d_model**.5,requires_grad=True)

        # Initial, encoding segment of our ConnTextUL model:
        # Instance of our Encoder module (defined above), for encoding orthography
        self.orthography_encoder = Encoder(d_model=d_model, nhead=nhead, num_layers=num_layers)
        # Instance of our Encoder module (defined above), for encoding phonology
        self.phonology_encoder = Encoder(d_model=d_model, nhead=nhead, num_layers=num_layers)

        # Criss-crossing orthography/phonology cross-attenion segment of ConnTextUL model
        self.gp_multihead_attention = pt.nn.MultiheadAttention(embed_dim=d_model, num_heads=nhead, batch_first=True)
        self.pg_multihead_attention = pt.nn.MultiheadAttention(embed_dim=d_model, num_heads=nhead, batch_first=True)

        # Segment of ConnTextUL model that mixes orthography/phonology representation
        self.transformer_mixer = Encoder(d_model=self.d_model, nhead=nhead, num_layers=num_layers)
        self.reduce = pt.nn.Linear(self.d_model,self.d_model)

        # Decoder segment of ConnTextUL model
        # Orthography component of Decoder segment
        self.orthography_decoder = Decoder(d_model=self.d_model, nhead=nhead, num_layers=num_layers)
        self.linear_orthography_decoder = pt.nn.Linear(self.d_model, self.vocab_sizes['orth_vocab_size'])
        # Phonology component of Decoder segment
        self.phonology_decoder = Decoder(d_model=self.d_model, nhead=nhead, num_layers=num_layers)
        self.linear_phonology_decoder = pt.nn.Linear(self.d_model, 2*(self.vocab_sizes['phon_vocab_size']-1))

    # Returns a size×size, strictly upper-triangular Boolean tensor
    def generate_triangular_mask(self, size, device):
        mask = pt.triu(pt.ones((size, size), dtype=pt.bool, device=device), 1)
        return mask

    def embed_tokens(self, tokens, mode='o'):
        assert mode in ['o','p']

        if mode == 'o':
           assert isinstance(tokens, pt.Tensor), "For orthographic embeddings, tokens must be a pytorch tensor of integers (indices of orthography_embedding)"
           return self.orthography_embedding(tokens) + self.position_embedding.weight[None,:tokens.shape[1]]
        # This is where we need to average the phonological embedding vectors
        else:
           try:
             isinstance(tokens, list)
           except:
             raise TypeError(f"For phonological vectors, tokens must be a list where each element is a pytorch tensor of integers (indices), but is type: {type(tokens)}")
           try:
             all(isinstance(token, pt.Tensor) for token in tokens)
           except:
             for token in tokens:
               print(f"type(token) = {type(token)}")
               print(f"token = {token}")
             raise TypeError("For phonological vectors, each element of the list must be a pytorch tensor of integers (indices)")
           # Here we average the embeddings for each feature in a phonological vector
           # Each row of indices will become of batch once we extract rows from the embedding matrix
           # So the size of the resulting 'output_embedding' tensor should be (batch_size, max_phon_len, d_model)
           output_embedding = pt.zeros((len(tokens), len(tokens[0]), self.d_model))
           for batch_num, batch in enumerate(tokens):
             for indx, tokes in enumerate(batch):
               # Here tokens should be a pytorch tensor of integers. It extracts the indicated rows from self.phonology_embedding
               avg_embedding = self.phonology_embedding(tokes).mean(axis=0)
               # Insert the resulting averaged embedding vector into the output_embedding tensor as a new row
               output_embedding[batch_num, indx, :] = avg_embedding
           return output_embedding + self.position_embedding.weight[None,:len(tokens[0])] 

    def embed(self, orthography, orthography_padding_mask, phonology, phonology_padding_mask):
        orthography, phonology = self.embed_tokens(orthography, 'o'), self.embed_tokens(phonology, 'p')

        orthography_encoding = self.orthography_encoder(orthography, src_key_padding_mask=orthography_padding_mask)
        try:
          phonology_encoding = self.phonology_encoder(phonology, src_key_padding_mask=phonology_padding_mask)
        except:
          print(f"phonology = {phonology}")
          print(f"phonology_padding_mask = {phonology_padding_mask}")
          raise
          

        # Query = orthography_encoding, Key = phonology_encoding
        gp_encoding = self.gp_multihead_attention(orthography_encoding, phonology_encoding, phonology_encoding,
                                                  key_padding_mask = phonology_padding_mask)[0]
        # Query = phonology_encoding, Key = orthography_encoding
        pg_encoding = self.pg_multihead_attention(phonology_encoding, orthography_encoding, orthography_encoding,
                                                  key_padding_mask = orthography_padding_mask)[0]

        # Concatenate outputs of cross-attention modules
        gp_pg = pt.cat((gp_encoding, pg_encoding), dim=1) + pt.cat((orthography_encoding, phonology_encoding), dim=1)
        # Concatenate padding masks
        gp_pg_padding_mask = pt.cat((orthography_padding_mask, phonology_padding_mask),dim=-1)

        # Here `global_embedding` is just a torch Parameter tensor.
        # If I'm not mistaken, the `repeat` method is dictating that the same
        # parameter tensor be applied to every entry in the batch.
        gp_pg = pt.cat((self.global_embedding.repeat(gp_pg.shape[0], 1, 1), gp_pg), dim=1)
        gp_pg_padding_mask = pt.cat((pt.zeros((gp_pg.shape[0], 1), device=gp_pg.device, dtype=pt.bool), gp_pg_padding_mask), dim=-1)
        mixed_encoding = self.transformer_mixer(gp_pg, src_key_padding_mask=gp_pg_padding_mask) 

        final_encoding = self.reduce(mixed_encoding[:,0]).unsqueeze(-2)

        return final_encoding


    def forward(self, orth_enc_input, orth_enc_pad_mask, orth_dec_input, orth_dec_pad_mask,
                      phon_enc_input, phon_enc_pad_mask, phon_dec_input, phon_dec_pad_mask):
        mixed_encoding = self.embed(orth_enc_input, orth_enc_pad_mask, 
                                    phon_enc_input, phon_enc_pad_mask)

        orth_ar_mask = self.generate_triangular_mask(orth_dec_input.shape[1], orth_dec_input.device)
        orth_dec_input = self.embed_tokens(orth_dec_input,'o')

        #print("Type of orth_dec_input = ", orth_dec_input.dtype)
        #print("Type of orth_ar_mask = ", orth_ar_mask.dtype)
        #print("Type of orth_pec_pad_mask = ", orth_dec_pad_mask.dtype)
        #print("Type of mixed_encoding = ", mixed_encoding.dtype)

        orth_output = self.orthography_decoder(tgt=orth_dec_input, 
                                               tgt_mask=orth_ar_mask,
                                               tgt_key_padding_mask=orth_dec_pad_mask,
                                               memory=mixed_encoding)

        phon_dec_input = self.embed_tokens(phon_dec_input,'p')
        phon_ar_mask = self.generate_triangular_mask(phon_dec_input.shape[1], phon_dec_input.device)

        #print("Type of phon_dec_input = ", phon_dec_input.dtype)
        #print("Type of phon_ar_mask = ", phon_ar_mask.dtype)
        #print("Type of phon_pec_pad_mask = ", phon_dec_pad_mask.dtype)
        phon_output = self.phonology_decoder(tgt=phon_dec_input, 
                                              tgt_mask=phon_ar_mask,
                                              tgt_key_padding_mask=phon_dec_pad_mask,
                                              memory=mixed_encoding)
        
        B, OC, E = orth_output.shape
        orth_output = orth_output.view(B*OC, E)
        orth_token_logits = self.linear_orthography_decoder(orth_output)
        B, PC, E = phon_output.shape
        phon_output = phon_output.view(B*PC, E)
        phon_token_logits = self.linear_phonology_decoder(phon_output)
        #print("(B, C))
        #print("orth_token_logits.shape = ", orth_token_logits.shape)
        #print("phon_token_logits.shape = ", phon_token_logits.shape)

        return {'orth': orth_token_logits.view(B, self.vocab_sizes['orth_vocab_size'], OC), 
                'phon': phon_token_logits.view(B, 2, PC, self.vocab_sizes['phon_vocab_size'] - 1)} # -1 because targets do not contain PAD


    def generate(self, orthography, orthography_mask, phonology, phonology_mask, max_new_tokens=21):
        self.eval()
        device = next(self.parameters()).device

        with pt.no_grad():
            prompt_encoding = self.embed(orthography, orthography_mask, phonology, phonology_mask)[0]

        mask = self.generate_triangular_mask(self.max_seq_len, device)

        generated_tokens = pt.zeros((2,prompt_encoding.shape[0]),dtype=pt.long)
        generated_embeddings = self.embed_tokens(generated_tokens)[:,:,None]
        generated_tokens = generated_tokens[:,:,None]

        dummy_mask = pt.zeros((1,15),device=device)
        dummy_mask[0,0] = 1

        for step in range(max_new_tokens):
            step_mask = mask[:step+1, :step+1]

            with pt.no_grad():
                orthography_token_logits = self.linear_orthography_decoder(self.orthography_decoder(generated_embeddings[0], prompt_encoding, tgt_mask=step_mask))
                phonology_token_logits = self.linear_phonology_decoder(self.phonology_decoder(generated_embeddings[1], prompt_encoding, tgt_mask=step_mask))

            last_token_logits = (gorthography_token_logits[:, -1, :], phonology_token_logits[:, -1, :])
            last_token_probs = (
                                    pt.softmax(last_token_logits[0], dim=-1),
                                    pt.softmax(last_token_logits[1], dim=-1)
                                )

            new_orthography_token = pt.multinomial(last_token_probs[0], num_samples=1)
            new_phonology_token = pt.multinomial(last_token_probs[1], num_samples=1)

            generated_tokens = pt.cat((generated_tokens, pt.stack((new_orthography_token,new_phonology_token), dim=0)), dim=2)

            generated_embeddings = pt.cat((generated_embeddings,pt.stack(
                                                (
                                                    self.embed_tokens(new_orthography_token,'o'),
                                                    self.embed_tokens(new_phonology_token,'p')
                                                  ),
                                            dim=0)), dim=2)
            
        return generated_tokens

    def size(self):

        param_num = 0
        param_size = 0
        for param in self.parameters():
            param_num += param.nelement()
            param_size += param.nelement() * param.element_size()
        buffer_num = 0
        buffer_size = 0
        for buffer in self.buffers():
            buffer_num += buffer.nelement()
            buffer_size += buffer.nelement() * buffer.element_size()

        param_num_all = param_num + buffer_num
        size_all_mb = round((param_size + buffer_size) / 1024**2,3)
        result = {'parameters': param_num_all,
                  'size in MB': size_all_mb}
        return result

## Training

#### Load Dataset

In [ ]:
ds = ConnTextULDataset()

### Training loop with WandB logging
The cell immediately below runs successive training loops for our model, logging data from each training loop run (called an "experiment") to `emelex`

**Note** In order for each WandB run to log a new instantiation of our model, we need to include the code from the above cell into the outermost, "experiment count" loop below.

In [ ]:
if pt.cuda.is_available():
   device = pt.device('cuda:0')
else:
   device = pt.device('cpu')

In [ ]:
exp_count = 1
num_epochs = 10
learning_rate = 5e-4
batch_size = 32
train_test_split = 0.8
cutpoint = int(train_test_split * len(ds))

In [ ]:
train_dataset_slices = []
for batch in range(math.ceil(cutpoint/batch_size)):
  train_dataset_slices.append(slice(batch*batch_size, min((batch+1)*batch_size, cutpoint)))

In [ ]:
len(train_dataset_slices)

2223

In [ ]:
val_dataset_slices = []
for batch in range(math.ceil((len(ds)-cutpoint)/batch_size)):
  val_dataset_slices.append(slice(cutpoint+batch*batch_size, min(cutpoint+(batch+1)*batch_size, len(ds))))

In [ ]:
len(val_dataset_slices)

556

In [ ]:
# A number for WandB:
n_steps_per_epoch = len(train_dataset_slices)

# Launch `exp_count` experiments, trying different dropout rates
for _ in range(exp_count):
    # 🐝 initialise a wandb run
    wandb.init(
        project="ConnTextUL_WandB_trial",
        config={
            "epochs": num_epochs,
            "batch_size": batch_size,
            "lr": learning_rate,
            #"dropout": random.uniform(0.01, 0.80),
            })
    
    # Copy your config 
    config = wandb.config

    pbar = tqdm.tqdm(range(num_epochs), position=0)

    gm = Model(len(ds.character_tokenizer), len(ds.phonology_tokenizer), d_model=512, nhead=8)

    gm.to(device)
    opt = pt.optim.Adam(gm.parameters(), learning_rate, weight_decay=1e-5)

    #Training
    example_ct = 0
    for epoch in pbar:
        gm.train()
        print("\nTraining Loop...")
        for step, batch_slice in enumerate(train_dataset_slices):
            batch = ds[batch_slice]
            orthography, phonology = batch['orthography'].to(device), batch['phonology'].to(device)
            logits = gm(orthography['enc_input_ids'], orthography['enc_pad_mask'].bool(),
                        orthography['dec_input_ids'], orthography['dec_pad_mask'].bool(),
                        phonology['enc_input_ids'], phonology['enc_pad_mask'].bool(),
                        phonology['dec_input_ids'], phonology['dec_pad_mask'].bool())
            if step == -1:
                print("---Predictions---\n")
                print("logits['orth'].shape = ", logits['orth'].shape)
                print("logits['orth'].dtype = ", logits['orth'].dtype)
                print("logits['phon'].shape = ", logits['phon'].shape)
                print("logits['phon'].dtype = ", logits['phon'].dtype)
                print("---Targets---\n")
                print("orthography['enc_input_ids'][1:].shape = ", orthography['enc_input_ids'][:,1:].shape)
                print("orthography['enc_input_ids'][1:].dtype = ", orthography['enc_input_ids'][:,1:].dtype)
                print("phonology['targets'].shape = ", phonology['targets'].shape) 
                print("phonology['targets'].dtype = ", phonology['targets'].dtype)                           
            orth_loss = pt.nn.CrossEntropyLoss(ignore_index=4)(logits['orth'] , orthography['enc_input_ids'][:,1:]) 
            phon_loss = pt.nn.CrossEntropyLoss(ignore_index=2)(logits['phon'], phonology['targets'])
            loss = orth_loss + phon_loss

            loss.backward()
            opt.step()
            opt.zero_grad()
            
            example_ct += len(batch['orthography'])
            metrics = {"train/orth_loss": orth_loss,
                       "train/phon_loss": phon_loss,
                       "train/train_loss": loss, 
                       "train/epoch": epoch, 
                       "train/example_ct": example_ct
                      }
            
            wandb.log(metrics)

        gm.eval()
        with pt.no_grad():
          print("\nValidation Loop...")
          for step, batch_slice in enumerate(val_dataset_slices):
              batch = ds[batch_slice]
              orthography, phonology = batch['orthography'].to(device), batch['phonology'].to(device)

              logits = gm(orthography['enc_input_ids'], orthography['enc_pad_mask'],
                          orthography['dec_input_ids'], orthography['dec_pad_mask'],
                          phonology['enc_input_ids'], phonology['enc_pad_mask'],
                          phonology['dec_input_ids'], phonology['dec_pad_mask'])
            
              val_loss = pt.nn.CrossEntropyLoss(ignore_index=4)(logits['orth'] , orthography['enc_input_ids'][:,1:]) 
              val_loss = val_loss + pt.nn.CrossEntropyLoss(ignore_index=2)(logits['phon'], phonology['targets'])              
              more_metrics = {"val/val_loss": val_loss}             
              wandb.log(more_metrics)

# 🐝 Close your wandb run 
wandb.finish()

train/epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/orth_loss,▅▅▅▄▃▃▄▄▄▂▅▃▄▂▁█▃▂▂▂▁▂▂▂▂▂▂▃▃▃▂▂▂▂▄▂▂▂▁▂
train/phon_loss,▄▅▄▄▄▅▃▃▃▄▄▃▄▄▃█▂▂▁▂▁▂▁▁▂▂▁▂▁▃▁▂▂▂▂▂▂▁▂▁
train/train_loss,▅▅▅▄▃▃▄▄▄▂▄▃▄▂▁█▃▂▂▂▁▂▂▂▂▂▁▃▃▃▂▂▂▂▄▂▂▁▁▂
train/epoch,0
train/example_ct,3196
train/orth_loss,1.88152
train/phon_loss,0.08636
train/train_loss,1.96788


  0%|          | 0/10 [00:00<?, ?it/s]


Training Loop...


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(



Validation Loop...


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:287: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:177.)
  output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:544: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._transformer_encoder_layer_fwd(
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:


Training Loop...

Validation Loop...


 20%|██        | 2/10 [2:35:58<10:43:35, 4826.95s/it]


Training Loop...


## Model Introspection

In [ ]:
gm = Model(len(ds.character_tokenizer), len(ds.phonology_tokenizer), d_model=512, nhead=8)
gm.size()

## Examples

Here we will walk through formatting the data for ingestion into the model, and examining the structure of the model output for ingestion into the CrossEntropy loss.




### Orthography -> Orthography
1) We will pass the words **whale** and **hop** through the model. This means we have a batch size of 2.

In [ ]:
inpt_words = ['whale', 'hop'] # Batch size 2

2) Focusing on the orthopgraphy -> orthography pathway first, let's explore the character-based tokenizer. Our final character vocab contains 7 unique characters from 2 words ('h', 'o', 'l', 'p', 'a', 'e', 'w') and 5 special tokens for a full vocab size of 12


In [ ]:
vocab = ['whale', 'hop'] # vocab size 7 unique characters and 5 special characters (12 tokens)
list_of_characters = set(''.join([c for word in vocab for c in word]))
character_tokenizer = CharacterTokenizer(list_of_characters)

In [ ]:
ORTH_VOCAB_SIZE = len(character_tokenizer.char_2_idx)
character_tokenizer.char_2_idx # Contains 14 elements

3) When we pass the inpt_words into the tokenizer, we receive a dictionary containing data formated for ingestion into the model.

In [ ]:
orth_data = character_tokenizer.encode(inpt_words)
print(f"--- tokens['enc_input_ids'] ---\n  shape: {orth_data['enc_input_ids'].shape}\n  {orth_data['enc_input_ids']}")
print(f"\n\n--- tokens['enc_pad_mask'] ---\n shape: {orth_data['enc_pad_mask'].shape}\n {orth_data['enc_pad_mask']}")
print(f"\n\n--- tokens['dec_input_ids'] ---\n  shape: {orth_data['dec_input_ids'].shape}\n  {orth_data['dec_input_ids']}")
print(f"\n\n--- tokens['dec_pad_mask'] ---\n shape: {orth_data['dec_pad_mask'].shape}\n {orth_data['dec_pad_mask']}")

We now extract the embedding vectors from the Embedding matrix for input into the transformer encoder layer. Output from the embedding matrix is a tensor ready to consumed by the transformer encoder. The shape is (batch_size, max_batch_len, d_model). Batch_size is specified by the number of words in the input list. max_batch_len, also known as context_length or block_size, is determined by the length of the longest word in the input list (plus 2). Some architectures manually set this to limit memory allocation for hardware constraints. Since our longest words will not begin to challenge our memory banks, we leave this value unbounded. The d_model parameter is a hyperparameter that we choose.

In [ ]:
D_MODEL = 10
orth_embed = pt.nn.Embedding(ORTH_VOCAB_SIZE, D_MODEL) # Creates Embedding Matrix of shape (ORTH_VOCAB_SIZE, d_model=10)
enc_input_tensors = orth_embed(orth_data['enc_input_ids']) # Grabs 2 batches of 2 rows from the Embedding Matrix
print(f"enc_input_tensors.shape = {enc_input_tensors.shape}") # (batch_size, max_batch_len, d_model)

Notice the context length for this input is 5 + 2 = 7, (because 5 is longest word in the batch _'whale'_ and we add the BOS and EOS tokens). This results in a full context length, or block_size, of 7. Next we pass the input tensors for the encoder **enc_input_tensors** along with their padding masks **input_data['enc_pad_mask']** into a transformer encoder layer

In [ ]:
encoder_layer = pt.nn.TransformerEncoderLayer(d_model=D_MODEL, nhead=1, batch_first=True)
enc_output = encoder_layer(enc_input_tensors, src_key_padding_mask=orth_data['enc_pad_mask'])
print(f"enc_output.shape = {enc_output.shape}") # (batch_size, context_length, d_model)

The **enc_output** will act as the memory in the orthography decoder. In our full implementation above, this will be the global mixed embedding vector. We may need to do a little reshaping to make things work there, but nothing complicated. Now we create the decoder instance and pass in the decoder input, the decoder pad mask, the memory, the memory mask, and the 'no look ahead' upper triangular mask.

In [ ]:
decoder_layer = pt.nn.TransformerDecoderLayer(d_model=10, nhead=1, batch_first=True)
dec_input_tensors = orth_embed(orth_data['dec_input_ids'])
context_length = dec_input_tensors.shape[1]
dec_look_ahead_mask = pt.triu(pt.ones((context_length, context_length), dtype=pt.bool, device='cpu'), 1)
dec_pad_mask = orth_data['dec_pad_mask']
mem_pad_mask = orth_data['enc_pad_mask']
out = decoder_layer(tgt=dec_input_tensors, 
                    tgt_key_padding_mask=dec_pad_mask,
                    tgt_mask=dec_look_ahead_mask,
                    memory=enc_output,
                    memory_key_padding_mask=mem_pad_mask)
print(f"out.shape = {out.shape}")

We next reshape the decoder output and pass it through a linear layer to predict the output tokens

In [ ]:
B, C, E = out.shape
out_reshaped = out.view(B*C, E)
linear = pt.nn.Linear(E, ORTH_VOCAB_SIZE)
logits = linear(out_reshaped)
print(f"logits.shape = {logits.shape}")
logits_reshaped = logits.view(B,ORTH_VOCAB_SIZE,C)
print(f"logits_reshaped.shape = {logits_reshaped.shape}")

In [ ]:
orth_data['enc_input_ids'][:,1:] # We slice from 1: becaues these are the targets of the input. i.e. the next character prediction.

In [ ]:
criterion = pt.nn.CrossEntropyLoss(ignore_index=4)
criterion(logits_reshaped, orth_data['enc_input_ids'][:,1:].long())

In [ ]:
-pt.log(pt.tensor(1/14)) # This is roughly the loss value we should expect from a well initialized model. CLOSE!

### Phonology -> Phonology

We now examine the p -> p pathway. In particular we look at how to format the input data for ingestion, the shapes of tensors along the computational graph, and then how to format the data for input into the loss function.

We begin by creating the phonological vectors for the corresponding input words. The vocab for phonology will be composed of the features of the phonological vectors. In otherwords, each of the elements in a single phonological vector (33 of them) will have its own row in the Phonology Embedding matrix.

In [ ]:
vocab = ['whale', 'hi']
phon_data = phonemizer.encode(vocab) # Produce the tokens for each phonological feature vector and decoder inputs/targets
PHON_VOCAB_SIZE = len(phonemizer) # The phonological vocab is 34 dimensional
print(f"PHON_VOCAB_SIZE = {PHON_VOCAB_SIZE}")

In [ ]:
phon_data.keys()

In [ ]:
phon_data

In [ ]:
# Create a phonological embedding matrix of size (PHON_VOCAB_SIZE, d_model)
phon_embed = pt.nn.Embedding(PHON_VOCAB_SIZE, D_MODEL)
print(f"phon_embed = ", phon_embed)

We now go through the process of creating the phonological input vectors. We need a single vembedding vector to represent the phonological feature vector, which is a distributed representation of phonological features. Therefore, each phonological feature that is present in the feature vector will be extracted from the phonology embedding matrix and averaged together, creating the single phonology embedding vector.

In [ ]:
# Create tensor of zeros of shape (batch_size, context_length, d_model)
phon_enc_input_tensors = pt.zeros((len(phon_data['enc_input_ids']), len(phon_data['enc_input_ids'][0]), D_MODEL))
print(f"phon_enc_input_tensors\n Shape: {phon_enc_input_tensors.shape}")
# We loop through each tensor in the enc_input_ids, extract all the embedding rows from the phon_embed and average them together
print(f"Loop over the lists of phonological feature tokens (elements in the batch)\n len(phon_data['enc_input_ids']) = {len(phon_data['enc_input_ids'])}")
for batch_num, batch_elem in enumerate(phon_data['enc_input_ids']):
  for indx, tokens in enumerate(batch_elem):
    # Here tokens should be a pytorch tensor of integers. It extracts the indicated rows from phon_embed
    avg_embedding = phon_embed(tokens).mean(axis=0)
    # We then insert the new average embedding vector into the phon_embedding_input tensor as a new row
    phon_enc_input_tensors[batch_num, indx, :] = avg_embedding

The final phonological features tensor for input into the encoder has the shape (batch_size, max_batch_length, d_model). Where once again, the batch_size corresponds to the number of words input to the model, the max_batch_length, also known as context_length or block_size, is the maximum number of phonological features active in any one feature vector within the entire batch, and d_model is a hyperparameter set during model creation.

In [ ]:
phon_enc_input_tensors.shape

In [ ]:
phon_enc_input_tensors

We must mask out the embedding vector associated with the padding feature vectors. Let us look at the mask created in the phonology tokenizer

In [ ]:
phon_data['enc_pad_mask'] # We see only the last token in the second word needs to be masked

In [ ]:
encoder_layer = pt.nn.TransformerEncoderLayer(d_model=D_MODEL, nhead=1, batch_first=True)
phon_enc_output = encoder_layer(phon_enc_input_tensors, src_key_padding_mask=phon_data['enc_pad_mask'])
print(f"phon_enc_output.shape = {phon_enc_output.shape}") # (batch_size, context_length, d_model)

In [ ]:
phon_enc_output

Now we create the decoder layer and pass all the decoder data created in the phonemizer through. First we must embed the decoder input, by calculating the average of the embedding vectors for each of the phonological features again

In [ ]:
# Create tensor of zeros of shape (batch_size, context_length, d_model)
context_length = len(phon_data['dec_input_ids'][0])
phon_dec_input_tensors = pt.zeros((len(phon_data['dec_input_ids']), context_length, D_MODEL))
# We loop through each tensor in the dec_input_ids, extract all the embedding rows from the phon_embed and average them together
for batch_num, batch_elem in enumerate(phon_data['dec_input_ids']):
  for indx, tokens in enumerate(batch_elem):
    # Here tokens should be a pytorch tensor of integers. It extracts the indicated rows from phon_embed
    avg_embedding = phon_embed(tokens).mean(axis=0)
    # We then insert the new average embedding vector into the phon_embedding_input tensor as a new row
    phon_dec_input_tensors[batch_num, indx, :] = avg_embedding
phon_dec_input_tensors.shape

Notice this resulting input tensor has a context_length or max_batch_length of 4, rather than 5 in the encoder input. That is because when decoding there are N-1 labels in a sequence of N tokens.

In [ ]:
phon_dec_input_tensors

In [ ]:
decoder_layer = pt.nn.TransformerDecoderLayer(d_model=D_MODEL, nhead=1, batch_first=True)
dec_look_ahead_mask = pt.triu(pt.ones((context_length, context_length), dtype=pt.bool, device='cpu'), 1)
dec_pad_mask = phon_data['dec_pad_mask']
mem_pad_mask = phon_data['enc_pad_mask']
phon_dec_out = decoder_layer(tgt=phon_dec_input_tensors, 
                    tgt_key_padding_mask=dec_pad_mask,
                    tgt_mask=dec_look_ahead_mask,
                    memory=phon_enc_output,
                    memory_key_padding_mask=mem_pad_mask)
print(f"out.shape = {phon_dec_out.shape}")

In [ ]:
# Now we evaluate the prediction of the target phonological vectors with the CrossEntropyLoss
B, C, E = phon_dec_out.shape
phon_dec_out_reshaped = phon_dec_out.view(B*C, E)
# The output of the linear layer is 2*C*(PHON_VOCAB_SIZE-1). The 2 is because for each phonological feature we are predicting 
# whether it is on or off, active or inactive. It is a binary random variable. The C is because we need to predict the entire
# feature vector for each 'phoneme' in the input, i.e. the context_length. The PHON_VOCAB_SIZE-1 is because our target vector
# that we are attempting to predict does not contain the PAD feature and it will not be predicted, therefore we subtract 1.
linear = pt.nn.Linear(E, 2*(PHON_VOCAB_SIZE-1)) 
logits = linear(phon_dec_out_reshaped)
print(f"logits.shape = {logits.shape}")
logits_reshaped = logits.view(B, 2, C, (PHON_VOCAB_SIZE-1))
print(f"logits_reshaped.shape = {logits_reshaped.shape}")

Before we calculate the loss, we verify that the output of our linear layer matches the shape of our target vector (minus the 2 dimensions for the probabilities of each class).

In [ ]:
phon_data['targets'].shape

In [ ]:
# Recall we set 2s in every rows that contained a padding vector. So we ask our loss function to ingore them
# with the ignore_index=2 parameter
criterion = pt.nn.CrossEntropyLoss(ignore_index=2)
criterion(logits_reshaped, phon_data['targets'].long())

In [ ]:
-pt.log(pt.tensor(1/2))

In [ ]:
print(pt.__version__)